<a href="https://colab.research.google.com/github/EricBojs/Barcoding-Nairobi/blob/master/4-Alternativa_modeller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4 Alternativa modeller $

## 4.1 För volatilitet $

I det här kompendiumet har vi tagit fram $\sigma$ genom en statisk modell där varje datapunkt är viktat jämnt. Grundantagandet som vi har gjort hela vår analys bygger på att $\sigma$ är kan ses ha ett konstant värde över den tid vi tar vårat stickprov. Volatiliteten skiftar dock över tid och därför finns det intresse av att minska vikten av äldre datapunkter.

Formeln som vi har utgått ifrån är den vanliga för att estimera standardavvikelse och viktar $m$ datapunkter jämt, alltså $\frac{1}{m}$: 

$$\hat{\sigma}^2_n = \frac{1}{m} \sum_{i=1}^{m} u^2_{n-i} : u_i = \ln \frac{S_i}{S_{i-1}}$$

Om vi vill vikta datapunker annorlunda behöver vi introducera en variabel $\alpha$:

$$\sigma^2_n = \sum_{i=1}^{m} u^2_{n-i} \cdot \alpha_i$$

där $\alpha$ bör sätta en större vikt andel på de senare datapunkerna, men alltid följer:
 
 $$\sum_{i=1}^{m} \alpha_i = 1.$$

En ytterligare förfining av den adaptiva modellen för $\sigma^2_n$ är att lägga till en långvarig varians, $V_L$, med viktningen $\gamma$ så att 

$$\sigma^2_n = V_L \cdot \gamma + \sum_{i=1}^{m} u^2_{n-i} \cdot \alpha_i \text{ där } \gamma + \sum_{i=1}^{m} \alpha_i = 1$$

Den långvariga variansen, $V_L$, bygger på en tro att variansen bör ligga kring något medel, men dra ifrån det om närliggande historik talar för det. 

### 4.1.1 EWMA-modellen

Denna modellen härleds ifrån den viktande modellen i det föregående avsnittet och tar större hänsyn till de senaste datapunkterna där $\alpha$ minskas exponentiellt ju längre bak i tiden vi går. Formeln lyder: 

$$\sigma^2 = \lambda\sigma^2_{n-1} + (1-\lambda)u^2_{n-1}$$

där $n$ är varje datapunkt. 
Ett högt värde på $\lambda < 1$ gör att vi får en mer långsam minskning i vår seriedata. I praxis brukar denna vara runt $0.94$.

Notera att formeln är rekursiv! 

Fördelarna med denna modell är:

* behövs relativt lite data 
* tar stor hänsyn till de senaste förändringarna i vårt dataset.


In [ ]:
def EwmaModelRekursiv(log_returns, lam):
  """
  rekursiva delen av EWMA-modellen
  """
  if len(log_returns)==1:
    return log_returns[0]**2
  else:
    return lam*EwmaModelRekursiv(log_returns[1:],lam)+(1-lam)*(log_returns[0]**2)

def EwmaModel(stickprov, lam):
  """
  EMWA-modellen utifrån en pandas-dataframe "stickprov" som har en kolumn
  'log_returns' redo.
  """
  log_returns = stickprov['log_returns'].to_numpy()
  sig_dy = np.sqrt(EwmaModelRekursiv(log_returns, lam))
  sig_yr = sig_dy*np.sqrt(252)
  return sig_yr

### 4.1.2 GARCH-modellen 

Till skillnad ifrån EMWA-modellen tar denna modell hänsyn till den långvariga medelvariansen, $V_L$. Formeln lyder:

$$\sigma^2_n = V_L\gamma + \alpha u^2_{n-1} + \beta\sigma^2_{n-1}$$

där $\gamma$ lägger en viktning på den långvariga medelvariansen, $\alpha$ lägger viktning på den kvadrerade förgående dagens avkastningen och $\beta$ på gårdagens varians. Och eftersom dessa variabler är viktade gäller:

$$\gamma + \alpha + \beta = 1$$

Om du kollar riktigt noggrant är EMWA-modellen bara ett specialfall av GARCH.

Fördelen med denna modell är att den också tar hänsyn till *mean reversion*, att saker det på sikt håller sig stabilt kring en viss volatilitet. 



In [ ]:
def GarchModelRekursiv(log_returns, V_L, gamma, alpha, beta):
  """
  Den rekursiva delen av formeln
  """
  if len(log_returns)==1:
    return log_returns[0]**2
  else:
    return V_L*gamma+alpha*(log_returns[0]**2)+beta*GarchModelRekursiv(log_returns[1:],V_L, gamma, alpha, beta)

def GarchModel(stickprov, V_L, gamma, alpha, beta):
  """
  Garch Model utifrån en pandas-dataframe "stickprov" som har en kolumn
  'log_returns' redo. Komihåg gamma+alpha+beta=1
  """
  log_returns = stickprov['log_returns'].to_numpy()       # skapa en vektor
  sig_dy = np.sqrt(GarchModelRekursiv(log_returns, V_L, gamma, alpha, beta))
  sig_yr = sig_dy*np.sqrt(252)
  return sig_yr

**Reflektionsfrågor**

1. Vilka antaganden bygger dessa modeller på?

2. Hur hanterar man det prognosfel som uppstår? 

3. Finns det omständigheter när den adaptiva modellen är att föredra framför den statiska och vice versa? 



## 4.2 För optioner \$

Det tog några sidor att gå igenom, men förhoppningsvis gav de föregående bladen någon form av insikt! För det är inte helt oanvändbar kunskap, köpoptioner fyller en viktig funktion. Investerare använder dessa, och liknande, rättigheter som en försäkring mot potentiella risker framtiden skymmer. Optioner är fyller en nyttig funktion, inte bara för investerare, utan också för industriverksamheter. En väldigt användbar option är något som kallas *valutaoptioner*.

En valutaoption ger dig rättighet till att köpa en bestämd mängd av en valuta i framtiden för en förutbestämt växelkurs. Prissättningen av en valutaoption görs på ett liknande sätt som på ett derivat med en aktie som underlag, och med viss ändring i formeln. Black scholes-modellen modifieras till följande: 

$$BS_\text{forex}(S_0,K,T,\sigma, r) = \Phi (d_1) S_0 \cdot e^{-r_f T} - \Phi (d_2) \cdot e^{-r_d T} K $$

där $d_1$ och $d_2$ ges av 

$$ d_1 = \frac{\ln{S_0/K}+(r_d-r_f + \sigma^2 /2) T }{\sigma \sqrt{T}}$$

$$d_2 = d_1 - \sigma  \sqrt{T} $$
 
variablerna som skiljer från den tidigare modellen är: 

* $S$ *spot exchange rate*, dagens växelkurs.

* $r_f$ *foreign  risk-free interest rate*, den utländska valutans riskfria ränta.

* $r_d$ *domestic risk-free interest rate* den inhemska valutans riskfria ränta.

In [ ]:
def black_scholes_forex(S, K, T, r_d, r_f, sigma):
  """
  Lämnas åt läsaren... Behövs i Kapitel 5!
  """
  return bs_pris